In [99]:
#import sys
#import pandas as pd
#import seaborn as sns
#from time import time
#from pyspark.sql import Row
#from pyspark.sql import SparkSession
#import matplotlib.pyplot as plt
#from pyspark.ml.feature import StringIndexer
#from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
#from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
#from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import MatrixFactorizationModel, Rating

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row


from pyspark.ml import Pipeline, PipelineModel
#from collections import namedtuple

In [59]:
# data = sc.textFile("hdfs://devenv/user/spark/mllib_als/ratings.dat")
# ratings = data.map(lambda l: l.split('::')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
# ratings.take(5)

[Rating(user=1, product=122, rating=5.0),
 Rating(user=1, product=185, rating=5.0),
 Rating(user=1, product=231, rating=5.0),
 Rating(user=1, product=292, rating=5.0),
 Rating(user=1, product=316, rating=5.0)]

In [89]:
data = spark.read.text("hdfs://devenv/user/spark/mllib_als/ratings.dat").rdd
#data.take(5)
parts = data.map(lambda row: row.value.split("::"))
#parts.take(5)
#ratingsRDD.take(5)
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), productId=int(p[1]), rating=float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)
ratings.show()

+---------+------+------+
|productId|rating|userId|
+---------+------+------+
|      122|   5.0|     1|
|      185|   5.0|     1|
|      231|   5.0|     1|
|      292|   5.0|     1|
|      316|   5.0|     1|
|      329|   5.0|     1|
|      355|   5.0|     1|
|      356|   5.0|     1|
|      362|   5.0|     1|
|      364|   5.0|     1|
|      370|   5.0|     1|
|      377|   5.0|     1|
|      420|   5.0|     1|
|      466|   5.0|     1|
|      480|   5.0|     1|
|      520|   5.0|     1|
|      539|   5.0|     1|
|      586|   5.0|     1|
|      588|   5.0|     1|
|      589|   5.0|     1|
+---------+------+------+
only showing top 20 rows



In [3]:
# users=ratings.map(lambda x:x.user).distinct()
# products = ratings.map(lambda x: x.product).distinct()

# print("Got "+ str(ratings.count())+" ratings from "+ str(users.count())+" users on "+str(products.count())+" products.")

Got 10000054 ratings from 69878 users on 10677 products.


In [90]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed=1111)
#train.take(5)

In [5]:
# rank = 12
# numIterations = 20
# lamda=0.01
# model = ALS.train(ratings, rank, numIterations, lamda)

In [103]:
#als = ALS(maxIter=5, regParam=0.01, userCol="user", itemCol="product", ratingCol="rating", coldStartStrategy="drop")
#als = ALS(userCol="user", itemCol="product", ratingCol="rating", coldStartStrategy="drop")
#model = als.fit(train)

#(trainingRatings, testRatings) = ratings.randomSplit([80.0, 20.0])
#als = ALS(userCol='user', itemCol='product', ratingCol='rating')

als = ALS(rank=10, maxIter=5, regParam=0.01, userCol="userId", itemCol="productId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

In [102]:
model.save("./als_model.data")

Py4JJavaError: An error occurred while calling o2382.save.
: java.io.IOException: Path ./als_model.data already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at 

In [6]:
model.userFeatures

<bound method MatrixFactorizationModel.userFeatures of <pyspark.mllib.recommendation.MatrixFactorizationModel object at 0x7f4c5fd538d0>>

In [92]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 0.818274108943101


In [94]:
# testdata = test.map(lambda p: (p[0], p[1]))

# predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
# ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

# MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
# print("Mean Squared Error = " + str(MSE))

In [101]:
# Save and load model
#Modelload = MatrixFactorizationModel.load("./als_model.data")

modelload=PipelineModel.load("./als_model.data")

IllegalArgumentException: 'requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.recommendation.ALSModel'